In [1]:
import os
import uuid
import json
import logging
from dotenv import load_dotenv

from dapr_agents import OpenAIChatClient
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from dapr_agents.tool import AgentTool

logging.basicConfig(level=logging.INFO)
load_dotenv()

True

In [ ]:
llm = OpenAIChatClient(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

def to_obj(x):
    return json.loads(x) if isinstance(x, str) else x

INFO:dapr_agents.llm.openai.client.base:Initializing Azure OpenAI client...
INFO:dapr_agents.llm.openai.client.azure:Using API key for authentication.


In [3]:
# ---- MCP Jupyter server env (REMOTE) ----
NOTEBOOK_NAME = f"mcp_{uuid.uuid4().hex[:8]}.ipynb"

ENV = {
    **os.environ,
    "MCP_JUPYTER_SESSION_MODE": "server",
    "MCP_JUPYTER_BASE_URL": os.getenv("JUPYTER_BASE_URL"),
    "MCP_JUPYTER_TOKEN": os.getenv("JUPYTER_TOKEN"),
    "MCP_JUPYTER_KERNEL_NAME": os.getenv("JUPYTER_KERNEL_NAME", "python3"),
    "MCP_JUPYTER_NOTEBOOK_PATH": os.getenv("JUPYTER_NOTEBOOK_PATH", NOTEBOOK_NAME),
}

# Adjust path to your MCP server entrypoint (where your server.py lives)
server_params = StdioServerParameters(
    command="uvx",
    args=[ "mcp-jupyter-notebook", "--transport", "stdio"],
    env=ENV,
)

In [4]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as s:
        await s.initialize()

        # Discover tools
        tools = await AgentTool.from_mcp_session(s)

        from dapr_agents import Agent

        notebook_agent = Agent(
            name="Jupyter Notebook Orchestrator",
            role="A notebook co-pilot that drives analysis inside a persistent Jupyter session.",
            goal=(
                "Use the active Jupyter notebook to complete tasks end-to-end: "
                "add clear markdown, execute Python code, and, when necessary, install Python packages. "
                "Reuse the same live session (kernel + document) so work builds on previous cells."
            ),
            instructions=[
                # Session & ordering
                "You are driving a live Jupyter notebook (persistent kernel + document).",
                "Cell order matters. Issue exactly ONE tool call at a time and wait for its result before the next.",
                "Append new cells; do not rewrite or reorder existing cells unless explicitly asked.",

                # Tools to use
                "Use `notebook.markdown.add(content)` for headings and short explanations.",
                "Use `notebook.code.run(content)` to append and execute Python code cells.",

                # Packages
                "If an import fails, call `notebook.packages.add([pip_names])` once, then retry the import.",
                "Package names are pip distributions (e.g., 'matplotlib', 'seaborn', 'plotly', 'requests').",

                # Notebook hygiene
                "Before a task, add a brief markdown heading describing what you'll do.",
                "Keep code cells small and focused—one idea per cell.",
                "When previewing data, show concise outputs (e.g., df.head(), shapes, key metrics).",
                "For plots, include a title, axis labels, and legends where helpful.",

                # Robustness
                "If an error occurs, add a short markdown note on the fix, then run a corrected code cell.",
                "Reuse variables and data created in earlier cells; reference them directly in later cells.",

                # Closure
                "End a logical section with a short markdown summary of what was produced."
            ],
            llm=llm,
            tools=tools,
            max_iterations=20,
        )

        QUESTION = """create a matplotlib bar and line chart examples in the notebook"""
        await notebook_agent.run(QUESTION)

INFO:dapr_agents.tool.executor:Tool registered: Notebook.Markdown.Add
INFO:dapr_agents.tool.executor:Tool registered: Notebook.Code.Run
INFO:dapr_agents.tool.executor:Tool registered: Notebook.Packages.Add
INFO:dapr_agents.tool.executor:Tool Executor initialized with 3 tool(s).


user:
create a matplotlib bar and line chart examples in the notebook

--------------------------------------------------------------------------------


INFO:dapr_agents.agents.agent.agent:Iteration 1/20 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Calling OpenAI ChatCompletion...


INFO:httpx:HTTP Request: POST https://bh25-shared-azfoundry-w3.cognitiveservices.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:ChatCompletion response received.


assistant:
Function name: Notebook.Markdown.Add (Call Id: call_MbSWeJwY5WhXvtdWqoovQ8Ho)
Arguments: {"content":"# Creating Example Matplotlib Bar and Line Charts\n\nIn this section, we'll create example visualizations using matplotlib to demonstrate how to make bar charts and line charts. This will include a simple dataset and clear labeled plots."}

--------------------------------------------------------------------------------


...


Notebook.Code.Run(tool) (Id: call_AHC06mTC5uwRLIZXoLMfcO8u):
{"ok":true,"status":"ok","execution_count":2,"cell_index":3,"stdout":"","stderr":"","outputs":[{"output_type":"display_data","data":{"text/plain":"<Figure size 800x400 with 1 Axes>","image/png":"..."},"metadata":{}}],"text_outputs":["<Figure size 800x400 with 1 Axes>"],"formatted_output":"<Figure size 800x400 with 1 Axes>","error_message":null,"elapsed_seconds":0.0695657730102539}

--------------------------------------------------------------------------------


INFO:dapr_agents.agents.agent.agent:Iteration 4/20 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Calling OpenAI ChatCompletion...


INFO:httpx:HTTP Request: POST https://bh25-shared-azfoundry-w3.cognitiveservices.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:ChatCompletion response received.


assistant:
I have added two example charts using matplotlib:

1. A bar chart with groups labeled as A to E with their respective values.
2. A line chart plotting points with a line connecting them, including labeled axes and a legend.

Both charts have titles, axis labels, and are clearly presented for easy understanding.

If you want, I can create combined bar and line charts or add more customization examples next.

--------------------------------------------------------------------------------


INFO:dapr_agents.agents.agent.agent:Agent conversation completed after 4 turns.
